<a href="https://colab.research.google.com/github/zhuzihan728/Image-Restore/blob/main/OneRestore_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup
- First, in the **Runtime** menu -> **Change runtime type**, make sure to have ```Hardware Accelerator = GPU```
- Clone repo and install dependencies.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

# git clone this repository
!git clone https://github.com/gy65896/OneRestore.git
%cd OneRestore

# # create new anaconda env
# !conda create -n onerestore python=3.7
# !conda activate onerestore

# # install pytorch (Take cuda 11.7 as an example to install torch 1.13)
# !pip install torch==1.13.0+cu117 torchvision==0.14.0+cu117 torchaudio==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu117

# install other packages
!pip install -r requirements.txt
!pip install gensim

Cloning into 'OneRestore'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 526 (delta 159), reused 111 (delta 111), pack-reused 361 (from 2)
Receiving objects: 100% (526/526), 51.27 MiB | 20.47 MiB/s, done.
Resolving deltas: 100% (261/261), done.
/content/OneRestore
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-3.0.1-py3-none-any.whl.metadata (10.0 kB)
Using cached pybind11-3.0.1-py3-none-any.whl (293 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp312-cp312-linux_x86_64.whl size=4498212 sha256=d8b6b81493ed1f581f2dc6c3113277c23a9dfc5a380efb6bde73e7e69dd52b7a
  Stored in directory: /root/.cache/pip/wheels/20/27/95/a7baf1b435f1cbde017cabdf1e9688526d2b0e929255a359c6
Successfully built

# 2. Download Pre-trained Models


In [12]:

# download ckpts
#put embedder_model.tar and onerestore_cdd-11.tar in ckpts folder
# embedder_model.tar https://1drv.ms/u/s!As3rCDROnrbLgqpnhSQFIoD9msXWOA?e=aUpHOT
# onerestore_cdd-11.tar https://1drv.ms/u/s!As3rCDROnrbLgqpmWkGBku6oj33efg?e=7yUGfN

embedder_path = '/content/drive/MyDrive/OneRestore/embedder_model.tar'
restorer_path = '/content/drive/MyDrive/OneRestore/onerestore_cdd-11.tar'

# 3. Inference

in test.py Change line 54 to `imwrite(out, f'{args.output}/{i}', normalize=True, value_range=(0, 1))`


In [ ]:
# !python test.py --embedder-model-path /content/drive/MyDrive/oneRestore/embedder_model.tar --restore-model-path /content/drive/MyDrive/oneRestore/onerestore_cdd-11.tar --input ./image/ --output ./output/ --concat

In [13]:
import torch
import torch.nn.functional as F
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage import img_as_ubyte
import cv2
import json
import os
from tqdm import tqdm
import numpy as np
from PIL import Image
from torchvision import transforms

# add model code path
import sys
sys.path.append('/content/OneRestore')
from utils.utils import load_restore_ckpt, load_embedder_ckpt

class EvalDataset:
    def __init__(self, corrupted_dir, original_dir, mask_dir, metadata_path,
                 im_size=None, transform=None):
        """
        :param corrupted_dir: path to corrupted images folder
        :param original_dir: path to original images folder
        :param mask_dir: path to mask images folder
        :param metadata_path: path to metadata.json
        :param im_size: target size (h, w) or None to keep original
        """
        self.corrupted_dir = corrupted_dir
        self.original_dir = original_dir
        self.mask_dir = mask_dir
        self.im_size = im_size
        self.transform = transform

        with open(metadata_path, 'r') as f:
            self.metadata = json.load(f)

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        meta = self.metadata[idx]

        corrupted = Image.open(os.path.join(self.corrupted_dir, meta['corrupted_image']))
        original = Image.open(os.path.join(self.original_dir, meta['original_image']))

        return corrupted, original, meta


class OneRestoreEvaluator:
    def __init__(self, embedder_path, restorer_path, eval_dataset, output_dir, prompt=None):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Load models
        print('> Loading OneRestore models...')
        self.embedder = load_embedder_ckpt(self.device, freeze_model=True, ckpt_name=embedder_path)
        self.restorer = load_restore_ckpt(self.device, freeze_model=True, ckpt_name=restorer_path)

        self.dataset = eval_dataset
        self.output_dir = f"{output_dir}_prompt_{str(prompt)}"
        self.prompt = prompt

        # Transform for embedder (224x224)
        self.transform_embedder = transforms.Compose([
            transforms.Resize([224, 224]),
            transforms.ToTensor()
        ])

        os.makedirs(os.path.join(self.output_dir, 'restored'), exist_ok=True)

    def rgb_to_y(self, img):
        """Convert RGB to Y channel - matches MATLAB rgb2ycbcr"""
        from skimage.color import rgb2ycbcr
        img_ycbcr = rgb2ycbcr(img)
        return img_ycbcr[:, :, 0]

    def evaluate(self):
        results = []

        with torch.no_grad():
            for idx in tqdm(range(len(self.dataset))):
                torch.cuda.ipc_collect()
                torch.cuda.empty_cache()

                # Load images
                corrupted, original, meta = self.dataset[idx]

                # Convert PIL to numpy
                corrupted_np = np.array(corrupted.convert('RGB'))
                original_np = np.array(original.convert('RGB'))

                # Prepare input for restorer (original size, normalized to [0,1])
                lq_re = torch.Tensor((corrupted_np / 255).transpose(2, 0, 1)).unsqueeze(0).to(self.device)

                # Prepare input for embedder (224x224)
                lq_em = self.transform_embedder(corrupted).unsqueeze(0).to(self.device)

                # Get text embedding
                if self.prompt is None:
                    text_embedding, _, [text] = self.embedder(lq_em, 'image_encoder')
                    print(f'Image {idx}: {text} degradation (auto-detected)')
                else:
                    text_embedding, _, [text] = self.embedder([self.prompt], 'text_encoder')
                    print(f'Image {idx}: {text} degradation (manual prompt)')

                # Restore image
                restored = self.restorer(lq_re, text_embedding)
                restored = torch.clamp(restored, 0, 1)

                # Convert to numpy
                restored_np = restored.permute(0, 2, 3, 1).cpu().detach().numpy()
                restored_np = img_as_ubyte(restored_np[0])

                # Calculate RGB metrics
                psnr_rgb = peak_signal_noise_ratio(original_np, restored_np, data_range=255)
                ssim_rgb = structural_similarity(original_np, restored_np, channel_axis=2, data_range=255)
                mae = np.mean(np.abs(original_np.astype(float) - restored_np.astype(float)))
                mse = np.mean((original_np.astype(float) - restored_np.astype(float)) ** 2)

                # Calculate Y channel metrics
                original_y = self.rgb_to_y(original_np)
                restored_y = self.rgb_to_y(restored_np)
                psnr_y = peak_signal_noise_ratio(original_y, restored_y, data_range=255)
                ssim_y = structural_similarity(original_y, restored_y, data_range=255)

                # Save restored image
                filename = meta['corrupted_image'].replace('_alpha', '_restored_alpha')
                cv2.imwrite(
                    os.path.join(self.output_dir, 'restored', filename),
                    cv2.cvtColor(restored_np, cv2.COLOR_RGB2BGR)
                )

                # Store results
                result = {
                    **meta,
                    'detected_degradation': text,
                    'psnr_rgb': float(psnr_rgb),
                    'ssim_rgb': float(ssim_rgb),
                    'psnr_y': float(psnr_y),
                    'ssim_y': float(ssim_y),
                    'mae': float(mae),
                    'mse': float(mse),
                    'restored_image': filename
                }
                results.append(result)

        # Save metrics
        with open(os.path.join(self.output_dir, 'eval_results.json'), 'w') as f:
            json.dump(results, f, indent=2)

        # Print summary
        print(f"\n=== Evaluation Results ===")
        print(f"Average PSNR (RGB): {np.mean([r['psnr_rgb'] for r in results]):.2f} dB")
        print(f"Average SSIM (RGB): {np.mean([r['ssim_rgb'] for r in results]):.4f}")
        print(f"Average PSNR (Y):   {np.mean([r['psnr_y'] for r in results]):.2f} dB")
        print(f"Average SSIM (Y):   {np.mean([r['ssim_y'] for r in results]):.4f}")
        print(f"Average MAE:        {np.mean([r['mae'] for r in results]):.2f}")
        print(f"Average MSE:        {np.mean([r['mse'] for r in results]):.2f}")

        # Per alpha range metrics
        avg_metrics = []
        alpha_ranges = list(set(str(r['alpha_range']) for r in results))
        alpha_ranges.sort()

        for alpha_range in alpha_ranges:
            alpha_results = [r for r in results if str(r['alpha_range']) == alpha_range]
            avg_metrics_alpha = {
                'alpha_range': alpha_range,
                'count': len(alpha_results),
                'avg_psnr (RGB)': float(np.mean([r['psnr_rgb'] for r in alpha_results])),
                'avg_ssim (RGB)': float(np.mean([r['ssim_rgb'] for r in alpha_results])),
                'avg_psnr (Y)': float(np.mean([r['psnr_y'] for r in alpha_results])),
                'avg_ssim (Y)': float(np.mean([r['ssim_y'] for r in alpha_results])),
                'avg_mae': float(np.mean([r['mae'] for r in alpha_results])),
                'avg_mse': float(np.mean([r['mse'] for r in alpha_results]))
            }
            avg_metrics.append(avg_metrics_alpha)
            print(f"\n======= Alpha {alpha_range} =======")
            print(f"PSNR (RGB): {avg_metrics_alpha['avg_psnr (RGB)']:.2f}")
            print(f"SSIM (RGB): {avg_metrics_alpha['avg_ssim (RGB)']:.4f}")
            print(f"PSNR (Y):   {avg_metrics_alpha['avg_psnr (Y)']:.2f}")
            print(f"SSIM (Y):   {avg_metrics_alpha['avg_ssim (Y)']:.4f}")
            print(f"MAE:        {avg_metrics_alpha['avg_mae']:.2f}")
            print(f"MSE:        {avg_metrics_alpha['avg_mse']:.2f}")

        avg_metrics_total = {
            'alpha_range': 'total',
            'count': len(results),
            'avg_psnr': float(np.mean([r['psnr_rgb'] for r in results])),
            'avg_ssim': float(np.mean([r['ssim_rgb'] for r in results])),
            'avg_psnr_y': float(np.mean([r['psnr_y'] for r in results])),
            'avg_ssim_y': float(np.mean([r['ssim_y'] for r in results])),
            'avg_mae': float(np.mean([r['mae'] for r in results])),
            'avg_mse': float(np.mean([r['mse'] for r in results]))
        }
        avg_metrics.append(avg_metrics_total)

        with open(os.path.join(self.output_dir, 'avg_metrics.json'), 'w') as f:
            json.dump(avg_metrics, f, indent=2)

        return results



eval_dataset = EvalDataset(
    corrupted_dir='/content/drive/MyDrive/eval_dataset/corrupted/',
    original_dir='/content/drive/MyDrive/image_test/',
    mask_dir='/content/drive/MyDrive/mask/',
    metadata_path='/content/drive/MyDrive/eval_dataset/metadata.json'
)

evaluator = OneRestoreEvaluator(
    embedder_path=embedder_path,
    restorer_path=restorer_path,
    eval_dataset=eval_dataset,
    output_dir='/content/drive/MyDrive/OneRestore/onerestore_results',
    prompt=None  # Set to None for auto-detection, or use 'low', 'haze', etc. for manual
)

results = evaluator.evaluate()

# # Test on a single image first
# corrupted, original, meta = eval_dataset[2]
# corrupted_np = np.array(corrupted.convert('RGB'))

# # Check input
# print(f"Input shape: {corrupted_np.shape}")
# print(f"Input range: [{corrupted_np.min()}, {corrupted_np.max()}]")

# # Run model
# input_ = torch.from_numpy(corrupted_np).float().div(255.).permute(2,0,1).unsqueeze(0).cuda()
# print(f"Model input shape: {input_.shape}")
# print(f"Model input range: [{input_.min()}, {input_.max()}]")

# with torch.no_grad():
#     restored = model(input_)
# restored = torch.clamp(restored, 0, 1)

# print(f"Model output shape: {restored.shape}")
# print(f"Model output range: [{restored.min()}, {restored.max()}]")

# # Visualize
# import matplotlib.pyplot as plt
# fig, axes = plt.subplots(1, 3, figsize=(15, 5))
# axes[0].imshow(corrupted_np)
# axes[0].set_title('Corrupted Input')
# axes[1].imshow(restored.permute(0,2,3,1).cpu().numpy()[0])
# axes[1].set_title('Model Output')
# axes[2].imshow(np.array(original.convert('RGB')))
# axes[2].set_title('Original')
# plt.show()


> Loading OneRestore models...
==> loading existing Embedder model: /content/drive/MyDrive/OneRestore/embedder_model.tar
Loaded embeddings from file ./utils/glove.6B.300d.txt torch.Size([12, 300])
==> loading existing OneRestore model: /content/drive/MyDrive/OneRestore/onerestore_cdd-11.tar
Number of OneRestore parameter: 5.98M


  0%|          | 0/300 [00:00<?, ?it/s]

Image 0: clear degradation (auto-detected)


  0%|          | 1/300 [00:04<20:49,  4.18s/it]

Image 1: haze degradation (auto-detected)


...

Image 298: haze_snow degradation (auto-detected)


100%|█████████▉| 299/300 [09:34<00:01,  1.83s/it]

Image 299: haze_snow degradation (auto-detected)


100%|██████████| 300/300 [09:36<00:00,  1.92s/it]


=== Evaluation Results ===
Average PSNR (RGB): 17.25 dB
Average SSIM (RGB): 0.7645
Average PSNR (Y):   18.82 dB
Average SSIM (Y):   0.7978
Average MAE:        36.21
Average MSE:        2659.67

======= Alpha [0.1, 0.5] =======
PSNR (RGB): 22.23
SSIM (RGB): 0.8858
PSNR (Y):   23.79
SSIM (Y):   0.9088
MAE:        20.79
MSE:        916.44

======= Alpha [0.5, 0.8] =======
PSNR (RGB): 16.33
SSIM (RGB): 0.7605
PSNR (Y):   17.90
SSIM (Y):   0.7971
MAE:        36.57
MSE:        2365.58

======= Alpha [0.8, 1.0] =======
PSNR (RGB): 13.17
SSIM (RGB): 0.6474
PSNR (Y):   14.76
SSIM (Y):   0.6874
MAE:        51.26
MSE:        4696.99
